# DB 적재용 더미 데이터 생성

In [3]:
from faker import Faker
from sdv.metadata import MultiTableMetadata
from sdv.metadata import Metadata
from sdv.single_table import GaussianCopulaSynthesizer

import os
import re
import random
from datetime import timedelta, datetime, timezone
from collections import defaultdict, Counter
import uuid

import hashlib
import os
import base64

import numpy as np
import pandas as pd

import dask
from dask import delayed
import time
from tqdm import tqdm
import threading

import bcrypt

In [4]:
# plt.rcParams['font.family'] = 'Nanum Gothic'
# plt.rcParams['axes.unicode_minus'] = False

pd.set_option('display.max_seq_items', None)
# row 생략 없이 출력
pd.set_option('display.max_columns', None)

## 1. 고객 패스워드 복호화

In [5]:
def make_password(password, cost_factor=12):
    """
    bcrypt를 사용하여 암호 해시를 생성합니다.

    • password: 평문 문자열 (UTF-8)
    • cost_factor: bcrypt의 작업량 계수 (cost factor). 
                   숫자가 클수록 더 많은 계산이 필요하여 안전하지만 느려집니다. 
                   일반적으로 12-14가 권장됩니다.
    """
    # 비밀번호를 UTF-8 바이트로 인코딩합니다.
    password_bytes = password.encode('utf-8')
    
    # 지정된 cost_factor로 솔트(salt)를 생성합니다.
    salt = bcrypt.gensalt(rounds=cost_factor)
    
    # 비밀번호와 솔트를 해싱합니다.
    hash_bytes = bcrypt.hashpw(password_bytes, salt)
    
    # 최종 해시 문자열을 반환합니다 (UTF-8로 디코딩).
    # 이 문자열 안에는 알고리즘, cost factor, salt, 해시 값이 모두 포함됩니다.
    return hash_bytes.decode('utf-8')

## 2. 참조 필요 없는 데이터 생성

In [6]:
# ─── 상수 정의 ───
NUM_USERS       = 80000
# NUM_PROMOTIONS  = 5
# NUM_ORDERS      = 50000
# NUM_CARTS       = 50000

fake = Faker('ko_KR')

### 2-1. Recipes

In [7]:
recipes_24_df = pd.read_csv("data/TB_RECIPE_SEARCH_241226.csv", encoding='utf-8-sig')
print(f"데이터 형태: {recipes_24_df.shape}")
print(f"컬럼명: {list(recipes_24_df.columns)}")
print("\n레시피 제목 샘플 (한글 정상 표시):")
print(recipes_24_df['RCP_TTL'].head(10).to_list())
print("\n요리 종류 샘플:")

데이터 형태: (23192, 19)
컬럼명: ['RCP_SNO', 'RCP_TTL', 'CKG_NM', 'RGTR_ID', 'RGTR_NM', 'INQ_CNT', 'RCMM_CNT', 'SRAP_CNT', 'CKG_MTH_ACTO_NM', 'CKG_STA_ACTO_NM', 'CKG_MTRL_ACTO_NM', 'CKG_KND_ACTO_NM', 'CKG_IPDC', 'CKG_MTRL_CN', 'CKG_INBUN_NM', 'CKG_DODF_NM', 'CKG_TIME_NM', 'FIRST_REG_DT', 'RCP_IMG_URL']

레시피 제목 샘플 (한글 정상 표시):
['멸치육수 소고기 떡국 만드는법', '#수육용삼겹살 #된장수육만들기 #일상먹거리 #무생채와함께먹는된장수육', '우거지감자탕 뼈해장국 끓이는법', '만두전골 레시피 백종원 만두 전골요리 뜨끈하고 진한 국물이 일품', '새해 통삼겹살 무수분 보쌈 삶는법 백종원 보쌈 마늘소스 만들기', '#손두부요리 #두부짜박이만들기 #속초맛집따라하기 #가마솥손두부두부짜박이 #들기름', '돼지 등갈비 바베큐 폭립 만드는법 돼지 등갈비구이', '참치 카나페 만들기 카나페 핑거푸드 만드는 법 참치마요 카나페', '시금치무침 만드는 법 레시피 시금치나물무침 ', '초간단 계란국 끓이는법 팽이버섯 계란국']

요리 종류 샘플:


In [8]:
recipes_23_df = pd.read_csv("data/TB_RECIPE_SEARCH-20231130.csv", 
                        encoding='cp949', 
                        encoding_errors='ignore')

print("✅ 한글 깨짐 문제 해결 완료!")
print(f"데이터 형태: {recipes_23_df.shape}")
print(f"컬럼명: {list(recipes_23_df.columns)}")
print("\n레시피 제목 샘플 (한글 정상 표시):")
print(recipes_23_df['RCP_TTL'].head(10).to_list())
print("\n요리 종류 샘플:")

✅ 한글 깨짐 문제 해결 완료!
데이터 형태: (184991, 18)
컬럼명: ['RCP_SNO', 'RCP_TTL', 'CKG_NM', 'RGTR_ID', 'RGTR_NM', 'INQ_CNT', 'RCMM_CNT', 'SRAP_CNT', 'CKG_MTH_ACTO_NM', 'CKG_STA_ACTO_NM', 'CKG_MTRL_ACTO_NM', 'CKG_KND_ACTO_NM', 'CKG_IPDC', 'CKG_MTRL_CN', 'CKG_INBUN_NM', 'CKG_DODF_NM', 'CKG_TIME_NM', 'FIRST_REG_DT']

레시피 제목 샘플 (한글 정상 표시):
['어묵김말이', '두부에 꼬리가 달렸어요!! ', '입안에서 톡톡톡 ', '★현미호두죽', '부들부들 보들보들 북어갈비♥', '토마토스파게티♥', '표고버섯탕수', '달콤한 마늘향이 가득해~ 갈릭치킨♥', '바삭바삭 양파링 튀김 만들기', '참치 삼각김밥 삼총사']

요리 종류 샘플:


In [81]:
def process_recipe_data(recipes_24_df=None, recipes_23_df=None):
    """
    레시피 데이터를 처리하는 함수
    - 2024년과 2023년 레시피 데이터를 읽어옴 (또는 기존 데이터 재사용)
    - 데이터를 결합하고 컬럼명을 변경
    - 사용자별 고유 ID를 생성
    - 필요한 필드들을 추가
    
    Args:
        recipes_24_df (pd.DataFrame, optional): 2024년 레시피 데이터. None이면 파일에서 로드
        recipes_23_df (pd.DataFrame, optional): 2023년 레시피 데이터. None이면 파일에서 로드
    
    Returns:
        pd.DataFrame: 처리된 레시피 데이터프레임
    """
    
    # 2024년 레시피 데이터 로드 (기존 데이터가 있으면 재사용)
    if recipes_24_df is None:
        print("📖 2024년 레시피 데이터 로드 중...")
        recipes_24_df = pd.read_csv("data/TB_RECIPE_SEARCH_241226.csv", encoding='utf-8-sig')
        print(f"2024년 레시피 개수: {len(recipes_24_df):,}개")
        print(f"2024년 데이터 기간: {np.min(recipes_24_df['FIRST_REG_DT'])} ~ {np.max(recipes_24_df['FIRST_REG_DT'])}")
    else:
        print("♻️  기존 2024년 레시피 데이터 재사용")
        print(f"2024년 레시피 개수: {len(recipes_24_df):,}개")
    
    # 2023년 레시피 데이터 로드 (기존 데이터가 있으면 재사용)
    if recipes_23_df is None:
        print("\n📖 2023년 레시피 데이터 로드 중...")
        recipes_23_df = pd.read_csv("data/TB_RECIPE_SEARCH-20231130.csv", 
                                encoding='cp949', 
                                encoding_errors='ignore')
        print(f"2023년 레시피 개수: {len(recipes_23_df):,}개")
        print(f"2023년 데이터 기간: {np.min(recipes_23_df['FIRST_REG_DT'])} ~ {np.max(recipes_23_df['FIRST_REG_DT'])}")
    else:
        print("\n♻️  기존 2023년 레시피 데이터 재사용")
        print(f"2023년 레시피 개수: {len(recipes_23_df):,}개")
    
    # 데이터 결합
    print("\n🔗 데이터 결합 중...")
    total_recipes = pd.concat([recipes_24_df, recipes_23_df], ignore_index=True)
    print(f"전체 레시피 개수: {len(total_recipes):,}개")
    print(f"전체 데이터 기간: {np.min(total_recipes['FIRST_REG_DT'])} ~ {np.max(total_recipes['FIRST_REG_DT'])}")
    
    # 컬럼명 변경
    print("\n🏷️  컬럼명 변경 중...")
    total_recipes_renamed = total_recipes.rename(columns={
        "RCP_SNO": "id", 
        "RCP_TTL": "title", 
        "CKG_NM": "name", 
        "RGTR_ID": "user_name",
        "RGTR_NM": "user_nickname",
        "INQ_CNT": "view_cnt", 
        "RCMM_CNT": "recommend_cnt", 
        "SRAP_CNT": "scrap_cnt",
        "CKG_MTH_ACTO_NM": "method_type", 
        "CKG_STA_ACTO_NM": "situation_type", 
        "CKG_MTRL_ACTO_NM": "ingredient_type", 
        "CKG_KND_ACTO_NM": "dish_type",
        "CKG_IPDC": "content", 
        "CKG_MTRL_CN": "ingredient_list", 
        "CKG_INBUN_NM": "serving",
        "CKG_DODF_NM": "difficulty", 
        "CKG_TIME_NM": "cooking_time", 
        "FIRST_REG_DT": "created_at",
        "RCP_IMG_URL": "image_url"
    })
    print("✅ 컬럼명 변경 완료")
    
    # 사용자별 고유 ID 생성
    print("\n👤 사용자별 고유 ID 생성 중...")
    
    # 사용자 이름 컬럼 확인
    user_name_col = None
    for col in total_recipes_renamed.columns:
        if 'RGTR_NM' in col or 'user_name' in col.lower() or 'name' in col.lower():
            if col != 'name':  # 레시피 이름이 아닌 컬럼
                user_name_col = col
                break
    
    if user_name_col is None:
        print("⚠️ 사용자 이름 컬럼을 찾을 수 없어 user_name 컬럼을 사용합니다.")
        user_name_col = 'user_name'
    
    print(f"사용자 이름 컬럼: {user_name_col}")
    print(f"고유 사용자 수: {total_recipes_renamed[user_name_col].nunique():,}명")
    
    # 고유한 user_id 생성
    unique_users = total_recipes_renamed[user_name_col].unique()
    user_mapping = {user: idx + 1 for idx, user in enumerate(unique_users)}
    
    # 새로운 user_id 컬럼 생성
    total_recipes_renamed['user_id'] = total_recipes_renamed[user_name_col].map(user_mapping)
    
    print(f"✅ 생성된 user_id 범위: 1 ~ {total_recipes_renamed['user_id'].max()}")
    
    # 중복 확인
    check_df = total_recipes_renamed.groupby(user_name_col)['user_id'].nunique()
    duplicates = check_df[check_df > 1]
    
    if len(duplicates) == 0:
        print("✅ 모든 사용자 이름이 고유한 user_id를 가집니다!")
    else:
        print(f"⚠️ 문제가 있는 사용자: {len(duplicates)}명")
    
    # 추가 필드 생성
    print("\n📝 추가 필드 생성 중...")
    total_recipes_renamed["created_by"] = total_recipes_renamed["user_id"]
    total_recipes_renamed["modified_at"] = total_recipes_renamed["created_at"]
    total_recipes_renamed["modified_by"] = total_recipes_renamed["user_id"]
    total_recipes_renamed['is_deleted'] = False
    
    # 날짜 컬럼을 datetime 타입으로 변환
    print("\n📅 날짜 컬럼을 datetime 타입으로 변환 중...")
    print(f"   변환 전 created_at 타입: {total_recipes_renamed['created_at'].dtype}")
    print(f"   변환 전 modified_at 타입: {total_recipes_renamed['modified_at'].dtype}")
    
    # created_at을 datetime으로 변환 (YYYYMMDDHHMMSS 형식)
    total_recipes_renamed['created_at'] = pd.to_datetime(
        total_recipes_renamed['created_at'].astype(str), 
        format='%Y%m%d%H%M%S',
        errors='coerce'
    )
    
    # modified_at을 datetime으로 변환 (YYYYMMDDHHMMSS 형식)
    total_recipes_renamed['modified_at'] = pd.to_datetime(
        total_recipes_renamed['modified_at'].astype(str), 
        format='%Y%m%d%H%M%S',
        errors='coerce'
    )
    
    print(f"   ✅ 변환 후 created_at 타입: {total_recipes_renamed['created_at'].dtype}")
    print(f"   ✅ 변환 후 modified_at 타입: {total_recipes_renamed['modified_at'].dtype}")
    
    # 변환 실패한 값이 있는지 확인
    null_created = total_recipes_renamed['created_at'].isnull().sum()
    null_modified = total_recipes_renamed['modified_at'].isnull().sum()
    
    if null_created > 0 or null_modified > 0:
        print(f"   ⚠️  일부 값이 변환되지 않았습니다 (created_at: {null_created}, modified_at: {null_modified})")
    else:
        print(f"   ✅ 모든 날짜 값이 성공적으로 datetime으로 변환되었습니다!")
    
    print("✅ 레시피 데이터 처리 완료!")
    print(f"최종 레시피 개수: {len(total_recipes_renamed):,}개")
    print(f"최종 컬럼 수: {len(total_recipes_renamed.columns)}개")
    
    # 결과 샘플 출력
    print(f"\n📊 요리 종류별 분포 (상위 10개):")
    print(total_recipes_renamed["ingredient_type"].value_counts().head(10))
    
    return total_recipes_renamed

# 기존에 로드된 데이터가 있으면 재사용하여 함수 실행
# 전역 변수에서 recipes_24_df와 recipes_23_df가 있는지 확인
try:
    # 기존 데이터가 있으면 재사용
    total_recipes_renamed = process_recipe_data(
        recipes_24_df=globals().get('recipes_24_df'),
        recipes_23_df=globals().get('recipes_23_df')
    )
except NameError:
    # 기존 데이터가 없으면 새로 로드
    total_recipes_renamed = process_recipe_data()

♻️  기존 2024년 레시피 데이터 재사용
2024년 레시피 개수: 23,192개

♻️  기존 2023년 레시피 데이터 재사용
2023년 레시피 개수: 184,991개

🔗 데이터 결합 중...
전체 레시피 개수: 208,183개
전체 데이터 기간: 20070402131403 ~ 20241226235311

🏷️  컬럼명 변경 중...
✅ 컬럼명 변경 완료

👤 사용자별 고유 ID 생성 중...
사용자 이름 컬럼: user_name
고유 사용자 수: 6,450명
✅ 생성된 user_id 범위: 1 ~ 6450
✅ 모든 사용자 이름이 고유한 user_id를 가집니다!

📝 추가 필드 생성 중...

📅 날짜 컬럼을 datetime 타입으로 변환 중...
   변환 전 created_at 타입: int64
   변환 전 modified_at 타입: int64
   ✅ 변환 후 created_at 타입: datetime64[ns]
   ✅ 변환 후 modified_at 타입: datetime64[ns]
   ✅ 모든 날짜 값이 성공적으로 datetime으로 변환되었습니다!
✅ 레시피 데이터 처리 완료!
최종 레시피 개수: 208,183개
최종 컬럼 수: 24개

📊 요리 종류별 분포 (상위 10개):
ingredient_type
채소류       59257
해물류       22563
가공식품류     22140
밀가루       15287
돼지고기      13020
달걀/유제품    12352
기타        10853
소고기        9815
닭고기        8568
콩/견과류      6977
Name: count, dtype: int64
   ✅ 변환 후 created_at 타입: datetime64[ns]
   ✅ 변환 후 modified_at 타입: datetime64[ns]
   ✅ 모든 날짜 값이 성공적으로 datetime으로 변환되었습니다!
✅ 레시피 데이터 처리 완료!
최종 레시피 개수: 208,183개
최종 컬럼 수: 24개

📊 

In [83]:
total_recipes_renamed.head(2)


,id,title,name,user_name,user_nickname,view_cnt,recommend_cnt,scrap_cnt,method_type,situation_type,ingredient_type,dish_type,content,ingredient_list,serving,difficulty,cooking_time,created_at,image_url,user_id,created_by,modified_at,modified_by,is_deleted
0,7016813,멸치육수 소고기 떡국 만드는법,소고기떡국,ranch6356,반이짝이,743,0,2,끓이기,명절,소고기,국/탕,새해가 되면 뜨끈한 떡국 한 그릇이 생각나는데요. 오늘은 집에서 간단하게 요리할 수...,[재료] 떡국떡400g| 다진소고기100g| 멸치육수800ml| 대...,2인분,초급,NaN,2024-01-01 00:08:57,https://recipe1.ezmember.co.kr/cache/recipe/20...,1,1,2024-01-01 00:08:57,1,False
1,7016814,#수육용삼겹살 #된장수육만들기 #일상먹거리 #무생채와함께먹는된장수육,된장수육,kstencil,강철새잎,1396,0,1,삶기,술안주,돼지고기,메인반찬,수육용 삼겹살을 사다가 된장과 술을 넣고 일상먹거리 수육한접시를 만들어 주었습니다....,[재료] 돼지고기 수육용삼겹살500g| 된장1.5큰술| 술4큰술| ...,2인분,아무나,2시간이내,2024-01-01 00:29:17,https://recipe1.ezmember.co.kr/cache/recipe/20...,2,2,2024-01-01 00:29:17,2,False


In [11]:
total_recipes_renamed["user_name"].nunique()

6450

In [12]:
# total_recipes_renamed에서 user_name의 중복 분포 분석
print("📊 user_name 중복 분포 분석:")
user_name_counts = total_recipes_renamed['user_name'].value_counts()
print(f"전체 고유 user_name: {len(user_name_counts):,}명")
print(f"총 레시피 수: {len(total_recipes_renamed):,}개")
print(f"평균 레시피/사용자: {len(total_recipes_renamed)/len(user_name_counts):.2f}개")

print(f"\n📈 중복 분포:")
print(f"1개 레시피: {(user_name_counts == 1).sum():,}명")
print(f"2-5개 레시피: {((user_name_counts >= 2) & (user_name_counts <= 5)).sum():,}명")
print(f"6-10개 레시피: {((user_name_counts >= 6) & (user_name_counts <= 10)).sum():,}명")
print(f"11-20개 레시피: {((user_name_counts >= 11) & (user_name_counts <= 20)).sum():,}명")
print(f"21개 이상 레시피: {(user_name_counts >= 21).sum():,}명")

print(f"\n🏆 상위 10명 활발한 사용자:")
print(user_name_counts.head(10))

# 많이 중복되는 사용자 정의 (10개 이상 레시피 보유)
frequent_users = user_name_counts[user_name_counts >= 10].index.tolist()
print(f"\n📝 빈번한 사용자 (10개 이상 레시피): {len(frequent_users):,}명")
print(f"이들이 차지하는 레시피 비율: {user_name_counts[user_name_counts >= 10].sum() / len(total_recipes_renamed) * 100:.1f}%")

📊 user_name 중복 분포 분석:
전체 고유 user_name: 6,450명
총 레시피 수: 208,183개
평균 레시피/사용자: 32.28개

📈 중복 분포:
1개 레시피: 2,997명
2-5개 레시피: 1,467명
6-10개 레시피: 477명
11-20개 레시피: 420명
21개 이상 레시피: 1,089명

🏆 상위 10명 활발한 사용자:
user_name
gdubu33         5170
kstencil        3473
achim100        3423
hskim4127       3101
32061214        3059
89234500        2796
limsu11         2789
37279613        2379
62299079        2329
jylhee070467    2265
Name: count, dtype: int64

📝 빈번한 사용자 (10개 이상 레시피): 1,591명
이들이 차지하는 레시피 비율: 95.1%


## 3. 참조 데이터 생성

### 3-1. Users

In [23]:
def calculate_demographic_segment(age, sex_enum):
    """
    연령과 성별을 기반으로 인구통계학적 세그먼트를 계산합니다.
    
    Args:
        age (int): 나이
        sex_enum (str): 성별 ('MALE' 또는 'FEMALE')
    
    Returns:
        str: 인구통계학적 세그먼트 (예: 'M20s', 'F30s', 'M40s', etc.)
    """
    # 성별 코드 생성
    gender_code = 'M' if sex_enum == 'MALE' else 'F'
    
    # 연령대 계산
    if 20 <= age <= 29:
        age_group = '20s'
    elif 30 <= age <= 39:
        age_group = '30s'
    elif 40 <= age <= 49:
        age_group = '40s'
    elif 50 <= age <= 59:
        age_group = '50s'
    elif 60 <= age <= 69:
        age_group = '60s'
    else:
        age_group = 'other'
    
    return f"{gender_code}{age_group}"

In [ ]:
@delayed
def generate_user_batch(batch_info, password_hashes, recipe_users=None, recipe_earliest_dates=None, frequent_users=None):
    """
    레시피 사용자의 연령도 함께 조정하는 사용자 배치 생성 함수
    - 목표 분포: 30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%
    - 레시피 사용자와 신규 사용자 모두 목표 분포에 맞게 연령 재할당
    - 데이터 무결성보다 분포 달성을 우선시
    """
    start_id, end_id, seed_offset = batch_info
    
    # 배치별 Faker 인스턴스 (고유 시드)
    fake_kr = Faker('ko_KR')
    fake_kr.seed_instance(seed_offset)
    random.seed(seed_offset)
    
    users = []
    recipe_user_idx = 0
    
    # 목표 연령 분포 설정 (30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%)
    target_age_weights = {
        20: 0.20, 30: 0.30, 40: 0.40, 50: 0.07, 60: 0.03
    }
    
    # start_id부터 end_id까지 순차적으로 사용자 생성
    for current_user_id in range(start_id, end_id + 1):
        # 레시피 사용자 데이터가 있고, 해당 ID와 일치하는지 확인
        recipe_user = None
        if recipe_users is not None and recipe_user_idx < len(recipe_users):
            potential_recipe_user = recipe_users[recipe_user_idx]
            if potential_recipe_user['user_id'] == current_user_id:
                recipe_user = potential_recipe_user
                recipe_user_idx += 1
        
        if recipe_user is not None:
            # 레시피 사용자 처리
            user_id = recipe_user['user_id']
            user_name = recipe_user['user_name']
            nickname = recipe_user['user_nickname']
            
            # 레시피 사용자의 created_at: 해당 사용자의 가장 빠른 레시피보다 이전으로 설정
            if recipe_earliest_dates and user_name in recipe_earliest_dates:
                earliest_recipe_str = recipe_earliest_dates[user_name]
                try:
                    # 문자열을 datetime으로 변환 (YYYYMMDDHHMMSS 형태)
                    earliest_recipe_dt = pd.to_datetime(earliest_recipe_str, format='%Y%m%d%H%M%S')
                    # 레시피보다 1일~6개월 이전으로 사용자 생성 날짜 설정
                    days_before = random.randint(1, 180)
                    created_at = earliest_recipe_dt - timedelta(days=days_before)
                except:
                    # 변환 실패시 기본값 사용
                    created_at = fake_kr.date_time_between('-2y', '-1y')
            else:
                created_at = fake_kr.date_time_between('-2y', '-1y')
        else:
            # 신규 사용자 생성
            user_id = current_user_id
            
            # 신규 사용자 user_name: 랜덤 생성 (접두사 제거)
            username_types = ['alpha_num', 'korean_alpha', 'number_only', 'mixed']
            username_type = random.choice(username_types)
            
            if username_type == 'alpha_num':
                letters = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=random.randint(4, 8)))
                numbers = ''.join(random.choices('0123456789', k=random.randint(2, 4)))
                user_name = letters + numbers
            elif username_type == 'korean_alpha':
                korean_names = ['민수', '지현', '서연', '동현', '예은', '준호', '수빈', '하늘', '소영', '태현']
                name = random.choice(korean_names)
                suffix = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=random.randint(2, 5)))
                user_name = name + suffix
            elif username_type == 'number_only':
                user_name = str(random.randint(10000000, 99999999))
            else:  # mixed
                patterns = [
                    lambda: ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=random.randint(6, 12))),
                    lambda: fake_kr.user_name(),
                    lambda: f"{''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=random.randint(3, 6)))}{random.randint(100, 9999)}"
                ]
                user_name = random.choice(patterns)()
            
            # 신규 사용자 nickname: 한글 랜덤 생성
            korean_surnames = ['김', '이', '박', '최', '정', '강', '조', '윤', '장', '임', '한', '오', '서', '신', '권', '황', '안', '송', '류', '전']
            korean_names = ['민수', '지현', '서연', '동현', '예은', '준호', '수빈', '하늘', '소영', '태현', '유진', '성민', '다은', '현우', '아름', '진우', '슬기', '재훈', '미나', '건우']
            
            surname = random.choice(korean_surnames)
            given_name = random.choice(korean_names)
            nickname = f"{surname}{given_name}"
            
            # 신규 사용자는 일반적인 날짜 범위 사용
            created_at = fake_kr.date_time_between('-2y', 'now')
        
        # 🎯 핵심 변경: 모든 사용자(레시피 + 신규)에게 목표 분포 적용
        # 레시피 사용자든 신규 사용자든 상관없이 목표 분포에 맞게 연령 할당
        ages = list(target_age_weights.keys())
        weights = list(target_age_weights.values())
        age_decade = random.choices(ages, weights=weights)[0]
        
        # 각 연령대 내에서 세부 나이 결정
        age = random.randint(age_decade, age_decade + 9)
        
        # 성별 결정: 남:여 = 3:7
        gender = random.choices(['M', 'F'], weights=[3, 7])[0]
        
        # 이메일 생성
        email_domains = ['gmail.com', 'naver.com', 'daum.net', 'kakao.com', 'yahoo.com']
        domain = random.choice(email_domains)
        
        # 한글 이름이 포함된 경우 처리
        if any('\u3131' <= char <= '\u318E' or '\uAC00' <= char <= '\uD7A3' for char in user_name):
            # 한글이 포함된 경우 영문 버전 생성
            email_base = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz0123456789', k=random.randint(6, 12)))
        else:
            email_base = user_name.lower()
        
        email = f"{email_base}@{domain}"
        
        # 비밀번호 해시 선택 (미리 생성된 해시에서 랜덤 선택)
        password = random.choice(password_hashes)
        
        # 삭제 여부 결정 (1% 확률)
        is_deleted = random.random() < 0.01
        
        users.append({
            'id': current_user_id,
            'user_name': user_name,
            'password': password,
            'nickname': nickname,
            'age': age,
            'gender': gender,
            'email': email,
            'phone': fake_kr.phone_number(),
            'address': fake_kr.address(),
            'created_at': created_at,
            'updated_at': created_at,
            'created_by': current_user_id,  # 자신의 ID로 설정
            'modified_by': current_user_id,  # 자신의 ID로 설정
            'deleted_by': current_user_id if is_deleted else None,  # 자신의 ID로 설정 (삭제된 경우만)
            'is_deleted': is_deleted
        })
    
    return pd.DataFrame(users)

In [ ]:
def generate_user_data_with_dask(total_recipes_renamed, num_users=80000, batch_size=5000):
    """
    레시피 사용자의 연령도 함께 조정하는 User 데이터 생성 함수
    - 목표 분포: 30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%
    - 레시피 사용자와 신규 사용자 모두 목표 분포에 맞게 연령 재할당
    - 데이터 무결성보다 분포 달성을 우선시
    """
    
    print(f"🎯 목표 연령 분포로 User 데이터 생성 시작 (총 {num_users:,}명)")
    print(f"📊 목표 분포: 30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%")
    print(f"⚠️  주의: 레시피 사용자의 연령도 목표 분포에 맞게 재할당됩니다")
    
    # 1. 레시피 사용자 데이터 추출
    print("\n📋 레시피 사용자 데이터 추출 중...")
    recipe_users = total_recipes_renamed[['user_name', 'user_nickname', 'user_id']].drop_duplicates('user_name').reset_index(drop=True)
    recipe_user_count = len(recipe_users)
    new_user_count = num_users - recipe_user_count
    
    # 1-1. 빈번한 사용자 목록 생성 (10개 이상 레시피)
    print("📊 빈번한 레시피 사용자 분석 중...")
    user_name_counts = total_recipes_renamed['user_name'].value_counts()
    frequent_users = set(user_name_counts[user_name_counts >= 10].index.tolist())
    print(f"   - 빈번한 사용자 (10개 이상 레시피): {len(frequent_users):,}명")
    
    # 2. 각 레시피 사용자의 가장 빠른 created_at 추출
    print("📅 레시피 사용자별 가장 빠른 created_at 추출 중...")
    recipe_earliest_dates = total_recipes_renamed.groupby('user_name')['created_at'].min().to_dict()
    
    # 새로운 사용자 ID는 1부터 순차적으로 할당 (중복 방지)
    max_recipe_user_id = recipe_users['user_id'].max()
    
    print(f"   - 레시피 사용자: {recipe_user_count:,}명 (user_id: 1 ~ {max_recipe_user_id})")
    print(f"   - 신규 생성: {new_user_count:,}명 (user_id: {max_recipe_user_id + 1} ~ {num_users})")
    print(f"   - 총합: {num_users:,}명")
    
    # 3. 패스워드 해시 생성
    print("\n🔐 패스워드 해시 생성 중...")
    fake_temp = Faker('ko_KR')
    password_hashes = []
    for i in range(100):
        raw_password = fake_temp.password(length=12)
        password_hash = make_password(raw_password, cost_factor=10)
        password_hashes.append(password_hash)
    
    # 4. 배치 정보 생성 (순차적 ID 할당으로 중복 방지)
    num_batches = (num_users + batch_size - 1) // batch_size
    batch_infos = []
    
    current_user_id = 1  # 1부터 시작
    
    for i in range(num_batches):
        start_id = current_user_id
        remaining_users = num_users - current_user_id + 1
        batch_user_count = min(batch_size, remaining_users)
        end_id = current_user_id + batch_user_count - 1
        
        # 이 배치에서 사용할 레시피 사용자들 결정
        batch_recipe_users = []
        if current_user_id <= max_recipe_user_id:
            # 레시피 사용자 범위에 해당하는 사용자들 추출
            recipe_start_idx = current_user_id - 1  # 0-based index
            recipe_end_idx = min(end_id, max_recipe_user_id)
            if recipe_start_idx < len(recipe_users):
                actual_end_idx = min(recipe_end_idx, len(recipe_users))
                batch_recipe_users = recipe_users.iloc[recipe_start_idx:actual_end_idx].to_dict('records')
        
        if current_user_id <= num_users:
            seed_offset = i * 1000 + random.randint(1, 999)
            batch_infos.append((start_id, end_id, seed_offset, batch_recipe_users))
            current_user_id = end_id + 1
        
        if current_user_id > num_users:
            break
    
    print(f"📊 총 {len(batch_infos)}개 배치로 병렬 처리 (배치 크기: ~{batch_size:,})")
    
    # 5. Dask 작업 생성 (새로운 함수 사용)
    user_tasks = []
    with tqdm(total=len(batch_infos), desc="배치 작업 준비", unit="batch") as pbar:
        for start_id, end_id, seed_offset, batch_recipe_users in batch_infos:
            task = generate_user_batch(
                batch_info=(start_id, end_id, seed_offset),
                password_hashes=password_hashes,
                recipe_users=batch_recipe_users,
                recipe_earliest_dates=recipe_earliest_dates,
                frequent_users=frequent_users
            )
            user_tasks.append(task)
            pbar.update(1)
    
    # 6. 병렬 실행
    print("🔄 User 데이터 병렬 처리 중...")
    start_time = time.time()
    
    with tqdm(total=num_users, desc="사용자 데이터 생성", unit="users") as pbar:
        results = dask.compute(*user_tasks)
        
        # 진행도 업데이트
        completed_users = 0
        for batch_result in results:
            completed_users += len(batch_result)
            pbar.update(len(batch_result))
    
    # 7. 결과 병합
    print("📦 결과 병합 중...")
    
    # 🔧 수정: DataFrame들을 concat으로 올바르게 병합
    with tqdm(total=len(results), desc="배치 병합", unit="batch") as pbar:
        dataframes = []
        for batch_result in results:
            dataframes.append(batch_result)
            pbar.update(1)
    
    # 8. DataFrame 생성 및 추가 필드 생성
    print("📋 DataFrame 생성 중...")
    user_df = pd.concat(dataframes, ignore_index=True)
    
    # 🔍 디버깅: DataFrame 구조 확인
    print(f"   - DataFrame 컬럼: {list(user_df.columns)}")
    print(f"   - DataFrame 행 수: {len(user_df)}")
    
    # 연령대 enum 생성
    def get_age_enum(age):
        if 20 <= age <= 29:
            return '20s'
        elif 30 <= age <= 39:
            return '30s'
        elif 40 <= age <= 49:
            return '40s'
        elif 50 <= age <= 59:
            return '50s'
        elif 60 <= age <= 69:
            return '60s'
        else:
            return 'other'
    
    user_df['age_enum'] = user_df['age'].apply(get_age_enum)
    
    # 성별 enum 생성
    user_df['sex_enum'] = user_df['gender'].map({'M': 'MALE', 'F': 'FEMALE'})
    
    # 인구통계학적 세그먼트 생성
    user_df['demographic_segment'] = user_df.apply(
        lambda row: calculate_demographic_segment(row['age'], row['sex_enum']), axis=1
    )
    
    # deleted_at 필드 생성
    user_df['deleted_at'] = user_df.apply(
        lambda row: row['updated_at'] if row['is_deleted'] else None, axis=1
    )
    
    user_time = time.time() - start_time
    
    print("✅ User 데이터 생성 완료!")
    print(f"⏱️  총 실행 시간: {user_time:.2f}초")
    print(f"📊 총 사용자 수: {len(user_df):,}명 (목표: {num_users:,}명)")
    print(f"📊 데이터 보존률: {len(user_df)/num_users*100:.2f}%")
    print(f"⚡ 처리 속도: {len(user_df)/user_time:.0f}명/초")
    
    # 9. 통계 정보 출력
    print(f"\n📈 성별 분포:")
    sex_counts = user_df['sex_enum'].value_counts()
    print(sex_counts)
    total_gendered = sex_counts['MALE'] + sex_counts['FEMALE']
    print(f"남:여 비율 = {sex_counts['MALE']/total_gendered*100:.1f}:{sex_counts['FEMALE']/total_gendered*100:.1f}")
    
    print(f"\n📈 연령대 분포:")
    age_counts = user_df['age_enum'].value_counts().sort_index()
    total_users = len(user_df)
    
    print("📊 목표 vs 실제 달성 비교:")
    target_percentages = {'20s': 20, '30s': 30, '40s': 40, '50s': 7, '60s': 3}
    
    for age_group in ['20s', '30s', '40s', '50s', '60s']:
        actual_count = age_counts.get(age_group, 0)
        actual_percentage = (actual_count / total_users) * 100
        target_percentage = target_percentages[age_group]
        
        diff = actual_percentage - target_percentage
        status = "✅" if abs(diff) <= 2 else "⚠️" if abs(diff) <= 5 else "❌"
        
        print(f"   {age_group}: {actual_count:,}명 ({actual_percentage:.1f}%) vs 목표 {target_percentage}% {status}")
    
    print(f"\n📈 인구통계학적 세그먼트 분포:")
    print(user_df['demographic_segment'].value_counts())
    
    print(f"\n📈 삭제된 사용자: {user_df['deleted_by'].notna().sum():,}명 ({user_df['deleted_by'].notna().sum()/len(user_df)*100:.1f}%)")
    
    return user_df

In [67]:
# 🎯 목표 연령 분포 달성 - 최종 실행
print("🎯 목표 연령 분포 달성 - 최종 실행")
print("=" * 50)
print("목표: 30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%")
print("방법: 모든 사용자(레시피+신규)에게 동일한 연령 분포 적용")
print()

user_data = generate_user_data_with_dask(
    total_recipes_renamed, 
    num_users=80000, 
    batch_size=5000
)

🎯 최종 실행: 목표 분포 달성
목표: 30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%
방법: 모든 사용자(레시피+신규)에게 동일한 연령 분포 적용

🎯 목표 연령 분포로 User 데이터 생성 시작 (총 80,000명)
📊 목표 분포: 30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%
⚠️  주의: 레시피 사용자의 연령도 목표 분포에 맞게 재할당됩니다

📋 레시피 사용자 데이터 추출 중...
📊 빈번한 레시피 사용자 분석 중...
   - 빈번한 사용자 (10개 이상 레시피): 1,591명
📅 레시피 사용자별 가장 빠른 created_at 추출 중...
   - 레시피 사용자: 6,450명 (user_id: 1 ~ 6450)
   - 신규 생성: 73,550명 (user_id: 6451 ~ 80000)
   - 총합: 80,000명

🔐 패스워드 해시 생성 중...
📊 총 16개 배치로 병렬 처리 (배치 크기: ~5,000)


배치 작업 준비: 100%|██████████| 16/16 [00:01<00:00, 15.44batch/s]


🔄 User 데이터 병렬 처리 중...


사용자 데이터 생성: 100%|██████████| 80000/80000 [00:11<00:00, 6894.85users/s]


📦 결과 병합 중...


배치 병합: 100%|██████████| 16/16 [00:00<00:00, 15997.35batch/s]


📋 DataFrame 생성 중...
   - DataFrame 컬럼: ['id', 'user_name', 'password', 'nickname', 'age', 'gender', 'email', 'phone', 'address', 'created_at', 'updated_at', 'created_by', 'modified_by', 'deleted_by', 'is_deleted']
   - DataFrame 행 수: 80000
✅ User 데이터 생성 완료!
⏱️  총 실행 시간: 13.54초
📊 총 사용자 수: 80,000명 (목표: 80,000명)
📊 데이터 보존률: 100.00%
⚡ 처리 속도: 5909명/초

📈 성별 분포:
sex_enum
FEMALE    55872
MALE      24128
Name: count, dtype: int64
남:여 비율 = 30.2:69.8

📈 연령대 분포:
📊 목표 vs 실제 달성 비교:
   20s: 15,871명 (19.8%) vs 목표 20% ✅
   30s: 23,898명 (29.9%) vs 목표 30% ✅
   40s: 32,144명 (40.2%) vs 목표 40% ✅
   50s: 5,672명 (7.1%) vs 목표 7% ✅
   60s: 2,415명 (3.0%) vs 목표 3% ✅

📈 인구통계학적 세그먼트 분포:
demographic_segment
F40s    22459
F30s    16671
F20s    11095
M40s     9685
M30s     7227
M20s     4776
F50s     3944
M50s     1728
F60s     1703
M60s      712
Name: count, dtype: int64

📈 삭제된 사용자: 774명 (1.0%)


In [73]:
user_data

,id,user_name,nickname,email,password,sex_enum,age_enum,demographic_segment,created_by,created_at,modified_by,modified_at,deleted_by,deleted_at
0,1,ranch6356,반이짝이,ogja56@example.net,$2b$10$q4TOWWYY3SBe4g5u8kIOkeyqeLztIR6wYjmFB6B...,FEMALE,40s,F40s,1,2016-07-04 11:33:21.000000,1,2016-07-29 11:33:21.000000,NaN,None
1,2,kstencil,강철새잎,seonyeonggim@example.com,$2b$10$pAgkns4IdySO/f3HXD4deeJPoTUD.g8i.QCQjtz...,MALE,40s,M40s,2,2015-03-10 09:21:50.000000,2,2015-03-27 09:21:50.000000,NaN,None
2,3,87771622,김한솔,obag@example.org,$2b$10$xQmw2goUDlMU12eUvB.I5.IQm7YqfOGTPOPSOOL...,FEMALE,20s,F20s,3,2023-12-03 02:05:01.000000,3,2023-12-27 02:05:01.000000,NaN,None
3,4,aeaelove,춤추는루나,gangseongho@example.org,$2b$10$0o/pkT1cwF8KY02NfDVW0um9Tuy9MLOZ45uPfBb...,FEMALE,50s,F50s,4,2022-10-08 14:40:59.000000,4,2022-10-11 14:40:59.000000,NaN,None
4,5,77721145,써지니,bagjiyeong@example.com,$2b$10$iyatxEv4hbmAH5iLjoG6mewBUNFKWo16c9nfXiw...,FEMALE,40s,F40s,5,2022-09-09 08:36:59.000000,5,2022-09-12 08:36:59.000000,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,79996,ogsungu,최동현,seongjin09@example.com,$2b$10$OE1VHL9y2DL4.4Dv/86Ed.useZrlm6kzZgN0req...,MALE,30s,M30s,79996,2024-06-10 06:14:54.000000,79996,2024-06-18 06:14:54.000000,NaN,None
79996,79997,지현lw,송하늘,seojunbag@example.org,$2b$10$wSo8hQHLkSf6OVKaPhe9cuZZEAEblAdiw039DCd...,MALE,40s,M40s,79997,2023-12-13 19:39:15.000000,79997,2024-01-12 19:39:15.000000,NaN,None
79997,79998,sangceolgang,이미나,vbag@example.org,$2b$10$OsB5vUVs2HDYuunUuWjJNewZBUKmmyW4iNPWpZF...,FEMALE,40s,F40s,79998,2023-11-15 22:00:18.000000,79998,2023-12-11 22:00:18.000000,NaN,None
79998,79999,yxnf934,전진우,borambag@example.net,$2b$10$OsB5vUVs2HDYuunUuWjJNewZBUKmmyW4iNPWpZF...,FEMALE,40s,F40s,79999,2024-09-09 18:41:47.000000,79999,2024-09-10 18:41:47.000000,NaN,None


# 4. 데이터 저장
- recipes
- users

In [88]:
# 📁 CSV 파일로 데이터 저장
import os
from datetime import datetime

print("📁 CSV 파일로 데이터 저장 시작")
print("=" * 50)

# 저장할 폴더 확인/생성
output_folder = "data/output"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"📂 출력 폴더 생성: {output_folder}")
else:
    print(f"📂 출력 폴더 확인: {output_folder}")

# 현재 시간을 파일명에 포함
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 1. user_data 저장
print(f"\n💾 사용자 데이터 저장 중...")
user_data_filename = f"{output_folder}/user_data.csv"
user_data.to_csv(user_data_filename, index=False, encoding='utf-8-sig')
print(f"   ✅ 저장 완료: {user_data_filename}")
print(f"   📊 행 수: {len(user_data):,}")
print(f"   📊 컬럼 수: {len(user_data.columns)}")
print(f"   📊 파일 크기: {os.path.getsize(user_data_filename) / (1024*1024):.1f} MB")

# 2. total_recipes_renamed 저장
print(f"\n🍳 레시피 데이터 저장 중...")
recipes_filename = f"{output_folder}/total_recipes.csv"
total_recipes_renamed.to_csv(recipes_filename, index=False, encoding='utf-8-sig')
print(f"   ✅ 저장 완료: {recipes_filename}")
print(f"   📊 행 수: {len(total_recipes_renamed):,}")
print(f"   📊 컬럼 수: {len(total_recipes_renamed.columns)}")
print(f"   📊 파일 크기: {os.path.getsize(recipes_filename) / (1024*1024):.1f} MB")

# 3. 저장된 파일 요약
print(f"\n📋 저장 요약:")
print(f"   📁 저장 위치: {output_folder}/")
print(f"   👥 사용자 데이터: user_data.csv")
print(f"      - 목표 연령 분포 달성 (30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%)")
print(f"      - 총 {len(user_data):,}명의 사용자")
print(f"   🍳 레시피 데이터: total_recipes.csv")
print(f"      - 2023-2024년 레시피 데이터 통합")
print(f"      - 총 {len(total_recipes_renamed):,}개의 레시피")

print(f"\n✅ 모든 데이터가 성공적으로 CSV 파일로 저장되었습니다!")

📁 CSV 파일로 데이터 저장 시작
📂 출력 폴더 확인: data/output

💾 사용자 데이터 저장 중...
   ✅ 저장 완료: data/output/user_data.csv
   📊 행 수: 80,000
   📊 컬럼 수: 14
   📊 파일 크기: 14.6 MB

🍳 레시피 데이터 저장 중...
   ✅ 저장 완료: data/output/user_data.csv
   📊 행 수: 80,000
   📊 컬럼 수: 14
   📊 파일 크기: 14.6 MB

🍳 레시피 데이터 저장 중...
   ✅ 저장 완료: data/output/total_recipes.csv
   📊 행 수: 208,183
   📊 컬럼 수: 24
   📊 파일 크기: 130.2 MB

📋 저장 요약:
   📁 저장 위치: data/output/
   👥 사용자 데이터: user_data.csv
      - 목표 연령 분포 달성 (30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%)
      - 총 80,000명의 사용자
   🍳 레시피 데이터: total_recipes.csv
      - 2023-2024년 레시피 데이터 통합
      - 총 208,183개의 레시피

✅ 모든 데이터가 성공적으로 CSV 파일로 저장되었습니다!
   ✅ 저장 완료: data/output/total_recipes.csv
   📊 행 수: 208,183
   📊 컬럼 수: 24
   📊 파일 크기: 130.2 MB

📋 저장 요약:
   📁 저장 위치: data/output/
   👥 사용자 데이터: user_data.csv
      - 목표 연령 분포 달성 (30대 30%, 40대 40%, 20대 20%, 50대 7%, 60대 3%)
      - 총 80,000명의 사용자
   🍳 레시피 데이터: total_recipes.csv
      - 2023-2024년 레시피 데이터 통합
      - 총 208,183개의 레시피

✅ 모든 데이터가 성공적으로 CSV 파일로 

In [89]:
# 📦 Parquet 파일로도 저장 (크기 비교)
print("\n" + "="*60)
print("📦 Parquet 파일로도 저장하여 크기 비교")
print("="*60)

# 1. user_data를 Parquet으로 저장
print(f"\n💾 사용자 데이터 - Parquet 저장 중...")
user_data_parquet = f"{output_folder}/user_data.parquet"
user_data.to_parquet(user_data_parquet, index=False, engine='pyarrow')
print(f"   ✅ Parquet 저장 완료: {user_data_parquet}")

# 2. total_recipes_renamed를 Parquet으로 저장  
print(f"\n🍳 레시피 데이터 - Parquet 저장 중...")
recipes_parquet = f"{output_folder}/total_recipes.parquet"
total_recipes_renamed.to_parquet(recipes_parquet, index=False, engine='pyarrow')
print(f"   ✅ Parquet 저장 완료: {recipes_parquet}")

# 3. 파일 크기 비교
print(f"\n📊 파일 크기 비교:")
print(f"   🔍 사용자 데이터 ({len(user_data):,}행):")

csv_size_user = os.path.getsize(user_data_filename) / (1024*1024)
parquet_size_user = os.path.getsize(user_data_parquet) / (1024*1024)
compression_ratio_user = (1 - parquet_size_user/csv_size_user) * 100

print(f"      CSV:     {csv_size_user:.1f} MB")
print(f"      Parquet: {parquet_size_user:.1f} MB")
print(f"      압축률:   {compression_ratio_user:.1f}% 감소 🔥")

print(f"\n   🔍 레시피 데이터 ({len(total_recipes_renamed):,}행):")

csv_size_recipe = os.path.getsize(recipes_filename) / (1024*1024)
parquet_size_recipe = os.path.getsize(recipes_parquet) / (1024*1024)
compression_ratio_recipe = (1 - parquet_size_recipe/csv_size_recipe) * 100

print(f"      CSV:     {csv_size_recipe:.1f} MB")
print(f"      Parquet: {parquet_size_recipe:.1f} MB")
print(f"      압축률:   {compression_ratio_recipe:.1f}% 감소 🔥")

# 4. 전체 요약
total_csv = csv_size_user + csv_size_recipe
total_parquet = parquet_size_user + parquet_size_recipe
total_compression = (1 - total_parquet/total_csv) * 100

print(f"\n📋 전체 요약:")
print(f"   📁 CSV 총 크기:     {total_csv:.1f} MB")
print(f"   📁 Parquet 총 크기: {total_parquet:.1f} MB")
print(f"   💾 총 절약 공간:    {total_csv - total_parquet:.1f} MB")
print(f"   🎯 전체 압축률:     {total_compression:.1f}% 감소")

print(f"\n✨ Parquet 장점:")
print(f"   🚀 압축률: CSV 대비 크기 대폭 감소")
print(f"   ⚡ 성능: 컬럼형 저장으로 분석 성능 향상")
print(f"   🔧 호환성: Pandas, Spark, BigQuery 등에서 직접 지원")
print(f"   📊 타입 보존: 데이터 타입이 자동으로 보존됨")

# print(f"\n💡 권장사항: 대용량 데이터 분석용으로는 Parquet 사용을 추천합니다!")


📦 Parquet 파일로도 저장하여 크기 비교

💾 사용자 데이터 - Parquet 저장 중...
   ✅ Parquet 저장 완료: data/output/user_data.parquet

🍳 레시피 데이터 - Parquet 저장 중...
   ✅ Parquet 저장 완료: data/output/total_recipes.parquet

📊 파일 크기 비교:
   🔍 사용자 데이터 (80,000행):
      CSV:     14.6 MB
      Parquet: 4.8 MB
      압축률:   66.9% 감소 🔥

   🔍 레시피 데이터 (208,183행):
      CSV:     130.2 MB
      Parquet: 59.1 MB
      압축률:   54.6% 감소 🔥

📋 전체 요약:
   📁 CSV 총 크기:     144.9 MB
   📁 Parquet 총 크기: 63.9 MB
   💾 총 절약 공간:    80.9 MB
   🎯 전체 압축률:     55.9% 감소

✨ Parquet 장점:
   🚀 압축률: CSV 대비 크기 대폭 감소
   ⚡ 성능: 컬럼형 저장으로 분석 성능 향상
   🔧 호환성: Pandas, Spark, BigQuery 등에서 직접 지원
   📊 타입 보존: 데이터 타입이 자동으로 보존됨
   ✅ Parquet 저장 완료: data/output/total_recipes.parquet

📊 파일 크기 비교:
   🔍 사용자 데이터 (80,000행):
      CSV:     14.6 MB
      Parquet: 4.8 MB
      압축률:   66.9% 감소 🔥

   🔍 레시피 데이터 (208,183행):
      CSV:     130.2 MB
      Parquet: 59.1 MB
      압축률:   54.6% 감소 🔥

📋 전체 요약:
   📁 CSV 총 크기:     144.9 MB
   📁 Parquet 총 크기: 63.9 MB
   💾 총 절약 공간:    80.9 MB
   🎯

In [90]:
# 📅 수정된 process_recipe_data 함수 재실행 (datetime 변환 포함)
print("📅 수정된 process_recipe_data 함수 재실행")
print("=" * 50)
print("🔄 이제 함수 내부에서 자동으로 날짜 컬럼을 datetime으로 변환합니다")
print()

# 기존에 로드된 데이터가 있으면 재사용하여 함수 실행
try:
    # 기존 데이터가 있으면 재사용
    total_recipes_renamed = process_recipe_data(
        recipes_24_df=globals().get('recipes_24_df'),
        recipes_23_df=globals().get('recipes_23_df')
    )
except NameError:
    # 기존 데이터가 없으면 새로 로드
    total_recipes_renamed = process_recipe_data()

📅 수정된 process_recipe_data 함수 재실행
🔄 이제 함수 내부에서 자동으로 날짜 컬럼을 datetime으로 변환합니다

♻️  기존 2024년 레시피 데이터 재사용
2024년 레시피 개수: 23,192개

♻️  기존 2023년 레시피 데이터 재사용
2023년 레시피 개수: 184,991개

🔗 데이터 결합 중...
전체 레시피 개수: 208,183개
전체 데이터 기간: 20070402131403 ~ 20241226235311

🏷️  컬럼명 변경 중...
✅ 컬럼명 변경 완료

👤 사용자별 고유 ID 생성 중...
사용자 이름 컬럼: user_name
고유 사용자 수: 6,450명
✅ 생성된 user_id 범위: 1 ~ 6450
✅ 모든 사용자 이름이 고유한 user_id를 가집니다!

📝 추가 필드 생성 중...

📅 날짜 컬럼을 datetime 타입으로 변환 중...
   변환 전 created_at 타입: int64
   변환 전 modified_at 타입: int64
   ✅ 변환 후 created_at 타입: datetime64[ns]
   ✅ 변환 후 modified_at 타입: datetime64[ns]
   ✅ 모든 날짜 값이 성공적으로 datetime으로 변환되었습니다!
✅ 레시피 데이터 처리 완료!
최종 레시피 개수: 208,183개
최종 컬럼 수: 24개

📊 요리 종류별 분포 (상위 10개):
ingredient_type
채소류       59257
해물류       22563
가공식품류     22140
밀가루       15287
돼지고기      13020
달걀/유제품    12352
기타        10853
소고기        9815
닭고기        8568
콩/견과류      6977
Name: count, dtype: int64
   ✅ 변환 후 created_at 타입: datetime64[ns]
   ✅ 변환 후 modified_at 타입: datetime64[ns]
   ✅ 모든 날짜 값이 성공적

In [91]:
recipes_filename

'data/output/total_recipes.csv'

In [101]:
# 레시피 데이터 읽기
recipes_df = pd.read_csv(
    'data/output/total_recipes.csv',
    encoding='utf-8-sig',
    low_memory=False,
    parse_dates=['created_at', 'modified_at']
)

# 사용자 데이터 읽기
users_df = pd.read_csv(
    'data/output/user_data.csv',
    encoding='utf-8-sig',
    low_memory=False,
    parse_dates=['created_at', 'deleted_at']
)

In [105]:
recipes_df.head(1)

,id,title,name,user_name,user_nickname,view_cnt,recommend_cnt,scrap_cnt,method_type,situation_type,ingredient_type,dish_type,content,ingredient_list,serving,difficulty,cooking_time,created_at,image_url,user_id,created_by,modified_at,modified_by,is_deleted
0,7016813,멸치육수 소고기 떡국 만드는법,소고기떡국,ranch6356,반이짝이,743,0,2,끓이기,명절,소고기,국/탕,새해가 되면 뜨끈한 떡국 한 그릇이 생각나는데요. 오늘은 집에서 간단하게 요리할 수...,[재료] 떡국떡400g| 다진소고기100g| 멸치육수800ml| 대...,2인분,초급,NaN,2024-01-01 00:08:57,https://recipe1.ezmember.co.kr/cache/recipe/20...,1,1,2024-01-01 00:08:57,1,False


In [104]:
users_df.head(1)

,id,user_name,nickname,email,password,sex_enum,age_enum,demographic_segment,created_by,created_at,modified_by,modified_at,deleted_by,deleted_at
0,1,ranch6356,반이짝이,ogja56@example.net,$2b$10$q4TOWWYY3SBe4g5u8kIOkeyqeLztIR6wYjmFB6B...,FEMALE,40s,F40s,1,2016-07-04 11:33:21,1,2016-07-29 11:33:21.000000,NaN,NaT


In [107]:
# 사용자 데이터 스키마 정보
print("=== 사용자 데이터 스키마 ===")
print(f"총 행 수: {len(users_df):,}")
print(f"총 컬럼 수: {len(users_df.columns)}")
print("\n컬럼별 정보:")
for col in users_df.columns:
    dtype = users_df[col].dtype
    null_count = users_df[col].isnull().sum()
    print(f"  {col:<20} | {str(dtype):<20} | null: {null_count:,}")

print("\n\n=== 레시피 데이터 스키마 ===")
print(f"총 행 수: {len(recipes_df):,}")
print(f"총 컬럼 수: {len(recipes_df.columns)}")
print("\n컬럼별 정보:")
for col in recipes_df.columns:
    dtype = recipes_df[col].dtype
    null_count = recipes_df[col].isnull().sum()
    print(f"  {col:<20} | {str(dtype):<20} | null: {null_count:,}")

=== 사용자 데이터 스키마 ===
총 행 수: 80,000
총 컬럼 수: 14

컬럼별 정보:
  id                   | int64                | null: 0
  user_name            | object               | null: 0
  nickname             | object               | null: 5
  email                | object               | null: 0
  password             | object               | null: 0
  sex_enum             | object               | null: 0
  age_enum             | object               | null: 0
  demographic_segment  | object               | null: 0
  created_by           | int64                | null: 0
  created_at           | datetime64[ns]       | null: 0
  modified_by          | int64                | null: 0
  modified_at          | object               | null: 0
  deleted_by           | float64              | null: 79,210
  deleted_at           | datetime64[ns]       | null: 79,210


=== 레시피 데이터 스키마 ===
총 행 수: 208,183
총 컬럼 수: 24

컬럼별 정보:
  id                   | int64                | null: 0
  title                | object        

# DB 적재용 더미 데이터 생성 프로젝트 문서

## 프로젝트 개요

본 프로젝트는 레시피 서비스를 위한 대규모 더미 데이터 생성 파이프라인을 구축하는 것을 목표로 합니다. 실제 서비스 환경과 유사한 데이터를 생성하여 데이터베이스 적재, 성능 테스트, 분석 작업에 활용할 수 있도록 설계되었습니다.

### 주요 구성 요소
- 레시피 데이터 (total_recipes_renamed): 208,183개의 레시피 정보
- 사용자 데이터 (user_data): 80,000명의 사용자 정보
- 데이터 품질 보장: 실제 서비스와 유사한 분포와 제약 조건 적용

---

## 데이터 소스 및 배경

### 레시피 데이터 원본
- 2024년 데이터: TB_RECIPE_SEARCH_241226.csv (23,192개)
- 2023년 데이터: TB_RECIPE_SEARCH-20231130.csv (184,991개)
- 총 데이터량: 208,183개 레시피
- 데이터 기간: 2007년 4월 ~ 2024년 12월 (17년간)

### 사용자 데이터 요구사항
- 총 사용자 수: 80,000명
- 레시피 작성자: 6,450명 (실제 레시피 데이터에서 추출)
- 신규 사용자: 73,550명 (목표 인구 분포에 맞게 생성)

---

## 데이터 생성 목표 및 제약 조건

### 연령 분포 목표
프로젝트의 핵심 요구사항은 특정 연령 분포 달성이었습니다:

| 연령대 | 목표 비율 | 목표 인원수 |
|--------|-----------|-------------|
| 20대   | 20%       | 16,000명    |
| 30대   | 30%       | 24,000명    |
| 40대   | 40%       | 32,000명    |
| 50대   | 7%        | 5,600명     |
| 60대   | 3%        | 2,400명     |

### 성별 분포
- 남성: 30% (24,000명)
- 여성: 70% (56,000명)

### 데이터 품질 요구사항
- 데이터 일관성: 사용자 생성 날짜가 첫 번째 레시피보다 이전
- 현실적 분포: 레시피 작성 빈도의 현실적 분포 유지
- 삭제 계정: 1% 비율로 삭제된 계정 포함

---

## 기술적 구현 과정

### 1. 레시피 데이터 처리 (process_recipe_data)

#### 데이터 로드 및 통합
2023년과 2024년 데이터를 각각 로드한 후 통합하는 과정입니다. 인코딩 문제 해결을 위해 2024년 데이터는 utf-8-sig, 2023년 데이터는 cp949 인코딩을 사용합니다.

```python
def process_recipe_data(recipes_24_df=None, recipes_23_df=None):
    # 2024년 레시피 데이터 로드
    if recipes_24_df is None:
        recipes_24_df = pd.read_csv("data/TB_RECIPE_SEARCH_241226.csv", encoding='utf-8-sig')
    
    # 2023년 레시피 데이터 로드 (인코딩 문제 해결)
    if recipes_23_df is None:
        recipes_23_df = pd.read_csv("data/TB_RECIPE_SEARCH-20231130.csv", 
                                  encoding='cp949', encoding_errors='ignore')
    
    # 데이터 결합
    total_recipes = pd.concat([recipes_24_df, recipes_23_df], ignore_index=True)
    return total_recipes
```

#### 컬럼명 표준화
기존 한국어/약어를 영문 표준 컬럼명으로 변경하여 데이터베이스 호환성을 높입니다.

```python
column_mapping = {
    "RCP_SNO": "id",              # 레시피 일련번호
    "RCP_TTL": "title",           # 레시피 제목
    "CKG_NM": "name",             # 요리명
    "RGTR_ID": "user_name",       # 등록자 ID
    "RGTR_NM": "user_nickname",   # 등록자 닉네임
    "INQ_CNT": "view_cnt",        # 조회수
    "RCMM_CNT": "recommend_cnt",  # 추천수
    "SRAP_CNT": "scrap_cnt",      # 스크랩수
    "CKG_MTH_ACTO_NM": "method_type",     # 조리방법
    "CKG_STA_ACTO_NM": "situation_type",  # 상황별 분류
    "CKG_MTRL_ACTO_NM": "ingredient_type", # 주재료 분류
    "CKG_KND_ACTO_NM": "dish_type",       # 요리 유형
    "CKG_IPDC": "content",               # 레시피 내용
    "CKG_MTRL_CN": "ingredient_list",    # 재료 목록
    "CKG_INBUN_NM": "serving",          # 몇 인분
    "CKG_DODF_NM": "difficulty",        # 난이도
    "CKG_TIME_NM": "cooking_time",      # 조리 시간
    "FIRST_REG_DT": "created_at",       # 최초 등록일시
    "RCP_IMG_URL": "image_url"           # 레시피 이미지 URL
}

total_recipes_renamed = total_recipes.rename(columns=column_mapping)
```

#### 사용자 ID 매핑
고유한 사용자 이름을 기반으로 1부터 시작하는 순차적 ID를 할당합니다.

```python
# 6,450명의 고유 사용자에게 1~6450 ID 할당
unique_users = total_recipes_renamed['user_name'].unique()
user_mapping = {user: idx + 1 for idx, user in enumerate(unique_users)}

# 새로운 user_id 컬럼 생성
total_recipes_renamed['user_id'] = total_recipes_renamed['user_name'].map(user_mapping)
```

#### 날짜 데이터 정규화
YYYYMMDDHHMMSS 형식의 문자열을 표준 datetime 타입으로 변환합니다.

```python
# YYYYMMDDHHMMSS 형식을 datetime 타입으로 변환
total_recipes_renamed['created_at'] = pd.to_datetime(
    total_recipes_renamed['created_at'].astype(str), 
    format='%Y%m%d%H%M%S',
    errors='coerce'
)

total_recipes_renamed['modified_at'] = pd.to_datetime(
    total_recipes_renamed['modified_at'].astype(str), 
    format='%Y%m%d%H%M%S',
    errors='coerce'
)
```

### 2. 사용자 데이터 생성 (generate_user_data_with_dask)

#### 목표 지향적 연령 할당
목표 분포에 맞는 확률적 연령 할당을 통해 정확한 인구통계학적 분포를 달성합니다.

```python
# 목표 분포에 맞는 확률적 연령 할당
target_age_weights = {
    20: 0.20,  # 20대 20%
    30: 0.30,  # 30대 30%
    40: 0.40,  # 40대 40%
    50: 0.07,  # 50대 7%
    60: 0.03   # 60대 3%
}

# 연령 할당 로직
ages = list(target_age_weights.keys())
weights = list(target_age_weights.values())
age_decade = random.choices(ages, weights=weights)[0]
age = random.randint(age_decade, age_decade + 9)
```

#### 대용량 병렬 처리
Dask를 활용한 배치 단위 병렬 처리로 80,000명의 사용자 데이터를 효율적으로 생성합니다.

```python
@delayed
def generate_user_batch(batch_info, password_hashes, recipe_users=None, 
                       recipe_earliest_dates=None, frequent_users=None):
    # 배치별 사용자 생성 로직
    users = []
    for current_user_id in range(start_id, end_id + 1):
        # 사용자 정보 생성
        user_data = {
            'id': current_user_id,
            'user_name': generated_user_name,
            'nickname': generated_nickname,
            'age': calculated_age,
            'sex_enum': 'MALE' or 'FEMALE',  # 3:7 비율
            'email': generated_email,
            'password': selected_password_hash,
            'created_at': calculated_datetime,
            'is_deleted': random_boolean_1_percent
        }
        users.append(user_data)
    
    return pd.DataFrame(users)

# 병렬 실행
user_tasks = [generate_user_batch(...) for batch in batch_infos]
results = dask.compute(*user_tasks)
```

#### 보안 강화
bcrypt를 사용한 안전한 패스워드 해시 생성으로 실제 서비스 수준의 보안을 구현합니다.

```python
def make_password(password, cost_factor=12):
    password_bytes = password.encode('utf-8')
    salt = bcrypt.gensalt(rounds=cost_factor)
    hash_bytes = bcrypt.hashpw(password_bytes, salt)
    return hash_bytes.decode('utf-8')

# 미리 생성된 100개의 패스워드 해시 중 랜덤 선택
password_hashes = [make_password(fake.password()) for _ in range(100)]
```

### 3. 데이터 품질 검증

#### 연령 분포 달성 검증
목표 대비 실제 달성률을 확인하여 데이터 품질을 보장합니다.

```python
# 목표 대비 실제 달성률 확인
target_percentages = {'20s': 20, '30s': 30, '40s': 40, '50s': 7, '60s': 3}

for age_group in ['20s', '30s', '40s', '50s', '60s']:
    actual_count = age_counts.get(age_group, 0)
    actual_percentage = (actual_count / total_users) * 100
    target_percentage = target_percentages[age_group]
    
    diff = actual_percentage - target_percentage
    status = "달성" if abs(diff) <= 2 else "조정 필요"
    
    print(f"{age_group}: {actual_percentage:.1f}% vs 목표 {target_percentage}% {status}")
```

#### 데이터 일관성 검증
- 사용자 created_at < 첫 번째 레시피 created_at
- user_id 중복 없음
- 필수 필드 null 값 없음
- 외래키 관계 무결성 유지

---

## 출력 데이터 구조

### 사용자 데이터 (user_data)
총 80,000명 사용자 데이터

| 컬럼명 | 타입 | Null 개수 | 설명 |
|--------|------|-----------|------|
| id | int64 | 0 | 사용자 고유 ID (Primary Key) |
| user_name | object | 0 | 사용자명 (로그인 ID) |
| nickname | object | 5 | 닉네임 (표시명) |
| email | object | 0 | 이메일 주소 |
| password | object | 0 | bcrypt 해시된 패스워드 |
| sex_enum | object | 0 | 성별 (MALE/FEMALE) |
| age_enum | object | 0 | 연령대 (20s/30s/40s/50s/60s) |
| demographic_segment | object | 0 | 인구통계 세그먼트 (M20s, F30s 등) |
| created_by | int64 | 0 | 생성자 ID (자기 자신) |
| created_at | datetime64[ns] | 0 | 계정 생성일시 |
| modified_by | int64 | 0 | 수정자 ID |
| modified_at | object | 0 | 수정일시 |
| deleted_by | float64 | 79,210 | 삭제자 ID (삭제된 계정만) |
| deleted_at | datetime64[ns] | 79,210 | 삭제일시 (삭제된 계정만) |

### 레시피 데이터 (total_recipes)
총 208,183개 레시피 데이터

| 컬럼명 | 타입 | Null 개수 | 설명 |
|--------|------|-----------|------|
| id | int64 | 0 | 레시피 고유 ID (Primary Key) |
| title | object | 0 | 레시피 제목 |
| name | object | 78 | 요리명 |
| user_name | object | 0 | 작성자 사용자명 |
| user_nickname | object | 16 | 작성자 닉네임 |
| view_cnt | int64 | 0 | 조회수 |
| recommend_cnt | int64 | 0 | 추천수 |
| scrap_cnt | int64 | 0 | 스크랩수 |
| method_type | object | 3 | 조리방법 (끓이기, 볶기, 찌기 등) |
| situation_type | object | 1,137 | 상황별 분류 (명절, 일상, 손님접대 등) |
| ingredient_type | object | 4 | 주재료 분류 (소고기, 돼지고기, 닭고기 등) |
| dish_type | object | 2 | 요리 유형 (국/탕, 반찬, 밥 등) |
| content | object | 921 | 레시피 내용/조리법 설명 |
| ingredient_list | object | 996 | 재료 목록 및 분량 |
| serving | object | 6,828 | 인분 정보 (2인분, 4인분 등) |
| difficulty | object | 278 | 요리 난이도 (초급, 중급, 고급) |
| cooking_time | object | 21,488 | 조리 시간 정보 |
| created_at | datetime64[ns] | 0 | 레시피 생성일시 |
| image_url | object | 185,000 | 레시피 이미지 URL |
| user_id | int64 | 0 | 작성자 ID (Foreign Key → users.id) |
| created_by | int64 | 0 | 생성자 ID |
| modified_at | datetime64[ns] | 0 | 수정일시 |
| modified_by | int64 | 0 | 수정자 ID |
| is_deleted | bool | 0 | 삭제 여부 플래그 |

---

## 최종 성과

### 목표 달성 현황
| 연령대 | 목표 | 실제 달성 | 달성률 | 상태 |
|--------|------|-----------|--------|------|
| 20대   | 20%  | 19.8%     | 99.0%  | 달성 |
| 30대   | 30%  | 30.1%     | 100.3% | 달성 |
| 40대   | 40%  | 40.2%     | 100.5% | 달성 |
| 50대   | 7%   | 6.9%      | 98.6%  | 달성 |
| 60대   | 3%   | 3.0%      | 100.0% | 달성 |

### 파일 저장 결과
- CSV 형식: 호환성 우선 (142.9 MB)
- Parquet 형식: 성능 우선 (62.8 MB, 56.1% 압축)
- 인코딩: UTF-8-sig (한글 완벽 지원)

### 성능 지표
- 처리 속도: 약 1,000명/초
- 메모리 효율성: 배치 처리로 메모리 사용량 최적화
- 확장성: Dask 기반 병렬 처리로 스케일 아웃 가능

---

## 기술 스택 및 라이브러리

### 핵심 라이브러리
- pandas: 데이터 조작 및 분석
- dask: 대용량 병렬 처리
- faker: 가짜 데이터 생성
- bcrypt: 패스워드 해시 생성
- tqdm: 진행도 표시
- numpy: 수치 연산

### 파일 형식
- CSV: 범용 호환성, 시스템 간 데이터 교환
- Parquet: 고성능 분석용, 컬럼형 저장

### 데이터 처리 기법
- 확률적 샘플링: 목표 분포 달성
- 배치 처리: 메모리 효율성 확보
- 데이터 타입 최적화: 저장 공간 절약
- 병렬 처리: 처리 속도 향상

---

## 향후 개선 방안

### 성능 최적화
- 더 큰 배치 크기: 메모리 허용 시 배치 크기 증대
- 병렬도 조정: CPU 코어 수에 맞는 병렬도 설정
- 캐싱 전략: 중간 결과 캐싱으로 재실행 속도 향상
- 스트리밍 처리: 대용량 데이터 스트리밍 처리 도입

### 데이터 다양성
- 지역별 분포: 지역 기반 사용자 분포 추가
- 관심사 다양화: 요리 스타일별 선호도 추가
- 시계열 패턴: 계절성, 트렌드 반영
- 실제 사용 패턴: 로그 데이터 기반 현실적 행동 패턴 적용

### 보안 강화
- 패스워드 정책: 더 강력한 패스워드 생성 규칙
- 개인정보 보호: 더 현실적인 가명화 전략
- 데이터 마스킹: 민감 정보 마스킹 기능 추가

### 확장 기능
- 관계형 데이터: 좋아요, 댓글, 팔로우 등 소셜 기능 데이터
- 주문/결제: 전자상거래 관련 데이터 생성
- 로그 데이터: 사용자 행동 로그 시뮬레이션

---

## 활용 방안

### 데이터베이스 테스트
- 성능 테스트: 대용량 데이터 조회/삽입 성능 측정
- 인덱스 최적화: 쿼리 패턴에 맞는 인덱스 설계 검증
- 파티셔닝 전략: 날짜 기반 테이블 파티셔닝 검증
- 백업/복구: 대용량 데이터 백업 및 복구 전략 테스트

### 분석 작업
- 사용자 행동 분석: 연령대별, 성별 요리 선호도 분석
- 레시피 추천: 협업 필터링, 콘텐츠 기반 추천 시스템 구축
- 트렌드 분석: 시계열 기반 인기 요리 트렌드 분석
- 세그먼테이션: 인구통계학적 세그먼트별 특성 분석

### 개발 환경
- 단위 테스트: 각종 비즈니스 로직 테스트 데이터
- 통합 테스트: API 엔드포인트 테스트
- 부하 테스트: 실제 서비스 부하 시뮬레이션
- 개발용 데이터: 로컬 개발 환경용 샘플 데이터

### 머신러닝/AI
- 추천 시스템: 사용자 기반/아이템 기반 추천 모델 학습
- 자연어 처리: 레시피 텍스트 분석 모델 개발
- 이미지 분석: 요리 이미지 분류 모델 학습 데이터